
# Classification


# 1.	Objectif : 

L’objectif de cet atelier est de découvrir la classification de documents texte à travers le classificateur SVM que nous allons appliquer sur un dataset contenant les news apparus sur le fil de presse Reuters en 1987.

Le datset peut être téléchargé à partir de ce lien : https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection. 

Le dataset est disponible egalement sur nltk :


In [2]:
import nltk
nltk.download('reuters')

[nltk_data] Downloading package reuters to /home/mohcin/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

Le dataset est disponible également sur scikitlearn 


In [ ]:
from sklearn.datasets import fetch_rcv1

# 2.	Chargement des données 

Nous considerons le dataset reuters à partir de NLTK.

In [3]:
from nltk.corpus import reuters
#recuperation du vocabulaire du corpus
vocabulaire=reuters.words()

#recuperation de toutes les categories
categories=reuters.categories()

#recuperation de tous les id des fichiers appartenant à une categorie bien determinée
ids_coffe=reuters.fileids("coffee")

#recuperation des mots contenus dans les documents d'une categorie bien determinee
coffe_words=reuters.words(reuters.fileids("coffee"))

#recuperation du texte brut des documents d'une categorie bien determinee
cofee_docs=reuters.raw(reuters.fileids("coffee")[0])

#recuperation de toutes les autres classe d'un document annoté avec une classe bien determinee
classes_Annotated_coffee=reuters.categories(reuters.fileids("coffee"))

#recuperer le dataset d'apprentissage
train_categories=[ reuters.categories(i) for i in reuters.fileids() if i.startswith('training/')]
train_documents = [reuters.raw(i) for i in reuters.fileids() if i.startswith('training/')]

#recuperer le dataset de test
test_documents=[reuters.raw(i)  for i in reuters.fileids() if i.startswith('test/')]
test_categories = [reuters.categories(i) for i in reuters.fileids() if i.startswith('test/')]

# recuperer tout le corpus
whole_docs=[reuters.raw(i)for i in reuters.fileids()]
whole_cats = [ reuters.categories(i) for i in reuters.fileids()]


# 3.	Prétraitements 



In [4]:
#TF-IDF
#fit(raw_documents[, y]): Learn vocabulary and idf from training set.
#fit_transform(raw_documents[, y]): Learn vocabulary and idf, return document-term matrix.
#transform(raw_documents): Transform documents to document-term matrix.

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = 'english')

vect_whole_docs = vectorizer.fit_transform(whole_docs)
vect_train_docs = vectorizer.transform(train_documents)
vect_test_docs = vectorizer.transform(test_documents)

#recuperer des labels uniques pour les categories
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_categories)
test_labels = mlb.transform(test_categories)
whole_labels = mlb.fit_transform(whole_cats)

print(whole_labels.shape)

(10788, 90)


# 4.	Classification avec SVM

In [5]:
#SVM
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

classifier_svm = OneVsRestClassifier(LinearSVC())
classifier_svm.fit(vect_train_docs,train_labels)
test_labels_predict=classifier_svm.predict(vect_test_docs)
print(classification_report(test_labels,test_labels_predict))
scores=classifier_svm.score(vect_test_docs,test_labels)
print(scores)

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       719
           1       1.00      0.43      0.61        23
           2       1.00      0.64      0.78        14
           3       0.95      0.60      0.73        30
           4       0.88      0.39      0.54        18
           5       0.00      0.00      0.00         1
           6       1.00      0.94      0.97        18
           7       1.00      0.50      0.67         2
           8       0.00      0.00      0.00         3
           9       0.96      0.96      0.96        28
          10       1.00      0.78      0.88        18
          11       0.00      0.00      0.00         1
          12       0.95      0.71      0.82        56
          13       1.00      0.50      0.67        20
          14       0.00      0.00      0.00         2
          15       0.92      0.43      0.59        28
          16       0.00      0.00      0.00         1
          17       0.91    

/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 5.	Classification  SVM avec cross validation

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_samples', 'recall_samples', 'f1_samples']
scores_svm = cross_validate(classifier_svm, vect_whole_docs, whole_labels, cv=3, scoring=scoring)
print(scores_svm['fit_time'].mean())
print(scores_svm['score_time'])
print(scores_svm['test_precision_samples'])
print(scores_svm['test_recall_samples'])
print(scores_svm['test_f1_samples'])




/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1.5013306140899658
[0.1101234  0.1072247  0.10513854]
[0.86598999 0.87977382 0.88399611]
[0.84546102 0.86121606 0.86284829]
[0.8475128  0.86216063 0.86600443]


/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 6.	Exercice


Q1. Réaliser le même processus avec le KNN et comparer les performances obtenues avec SVM.

Q2. Réaliser le même processus avec une méthode ensembliste et comparer les performances obtenues avec KNN et SVM. 

NB: There is no multi-label version of AdaBoost and the label should be the shape of samples number. 



## Q1 : KNN

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier

classifier_knn = OneVsRestClassifier(KNeighborsClassifier())
scoring = ['precision_samples', 'recall_samples', 'f1_samples']
scores_knn = cross_validate(classifier_knn, vect_whole_docs, whole_labels, cv=3, scoring=scoring)
print(scores_knn['fit_time'])
print(scores_knn['score_time'])
print(scores_knn['test_precision_samples'])
print(scores_knn['test_recall_samples'])
print(scores_knn['test_f1_samples'])

/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.32884479 0.32779479 0.43443036]
[103.13968492 116.36768055 131.13470197]
[0.80500093 0.81952368 0.82848999]
[0.79271524 0.80640769 0.81499286]
[0.7895614  0.80409413 0.81370121]


/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Q2 : LGBMClassifier

In [9]:
from lightgbm import LGBMClassifier

classifier_lgbm = OneVsRestClassifier(LGBMClassifier())
scoring = ['precision_samples', 'recall_samples', 'f1_samples']
scores_lgbm = cross_validate(classifier_lgbm, vect_whole_docs, whole_labels, cv=3, scoring=scoring)
print(scores_lgbm['fit_time'])
print(scores_lgbm['score_time'])
print(scores_lgbm['test_precision_samples'])
print(scores_lgbm['test_recall_samples'])
print(scores_lgbm['test_f1_samples'])

/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohcin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` par

[216.5671947  224.40684271 229.89791465]
[0.86417866 1.01126099 0.91599703]
[0.79261889 0.81948086 0.80516281]
[0.85709963 0.87537509 0.86848951]
[0.80489487 0.83033783 0.81912777]


## Comparaison

In [11]:
import pandas as pd

score_svm_list = [
    scores_svm['fit_time'].mean(),
    scores_svm['score_time'].mean(),
    scores_svm['test_precision_samples'].mean(),
    scores_svm['test_recall_samples'].mean(),
    scores_svm['test_f1_samples'].mean(),
]

score_knn_list = [
    scores_knn['fit_time'].mean(),
    scores_knn['score_time'].mean(),
    scores_knn['test_precision_samples'].mean(),
    scores_knn['test_recall_samples'].mean(),
    scores_knn['test_f1_samples'].mean(),
]

score_lgbm_list = [
    scores_lgbm['fit_time'].mean(),
    scores_lgbm['score_time'].mean(),
    scores_lgbm['test_precision_samples'].mean(),
    scores_lgbm['test_recall_samples'].mean(),
    scores_lgbm['test_f1_samples'].mean(),
]

score_df = pd.DataFrame(list(zip(score_svm_list, score_knn_list, score_lgbm_list)),
        ['fit_time','score_time','test_precision_samples','test_recall_samples','test_f1_samples'],
        ["SVM","KNN","LGBM"])
score_df.head()

,SVM,KNN,LGBM
fit_time,1.501331,0.363690,223.623984
score_time,0.107496,116.880689,0.930479
test_precision_samples,0.876587,0.817672,0.805754
test_recall_samples,0.856508,0.804705,0.866988
test_f1_samples,0.858559,0.802452,0.818120


on remarque que SVM nous donne en général des score plus elevés